### Laden der Bibliotheken

In [10]:
import pandas as pd
import numpy as np
import os

# Import des Datensatzes
dataf = pd.read_csv("https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/dataf.csv")
dataf.head()  # Erste Zeilen anzeigen

,Datum,KielerWoche,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Beschreibung,FerienSH,Feiertag,Uhrzeit,Heim_Auswärts,Umschlag,Weihnachtsmarkt,Regen,Wochentag_MDMDFSS,Wochenende,Jahreszeit_FSHW,Temperatur_Kategorie
0,2012-01-01,NaN,NaN,NaN,8.0,9.8250,14.0,58.0,leichter Spruehregen mit Regen,1.0,1.0,NaN,NaN,NaN,NaN,1,6,1,4,Niedrig
1,2012-01-02,NaN,NaN,NaN,7.0,7.4375,12.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0,0,0,4,Niedrig
2,2012-01-03,NaN,NaN,NaN,8.0,5.5375,18.0,63.0,durchgehend maessiger Regen,1.0,0.0,NaN,NaN,NaN,NaN,1,1,0,4,Niedrig
3,2012-01-04,NaN,NaN,NaN,4.0,5.6875,19.0,80.0,leichter Regenschauer,1.0,0.0,NaN,NaN,NaN,NaN,1,2,0,4,Niedrig
4,2012-01-05,NaN,NaN,NaN,6.0,5.3000,23.0,80.0,leichter Regenschauer,1.0,0.0,NaN,NaN,NaN,NaN,1,3,0,4,Niedrig


### Daten in das richtige Format bringen und kodieren, damit das neuronale Netz am Ende was damit anfangen kann

In [14]:
print(dataf.dtypes)

# sichergehen dass das Datum richtig formatiert ist
dataf['Datum'] = pd.to_datetime(dataf['Datum'])

# Das encoded die Warengruppe über mehrere Spalten
pivot = dataf.pivot_table(index='Datum', columns='Warengruppe', values='Umsatz').reset_index()
print(pivot.head())

# Features aus dem Datensatz herausziehen (hier nur beispielhaft, muss erweitert werden), die ins neuronale Netz eingehen sollen (es scheinen alle am Ende Kategorial zu sein bei uns)
categorical_features = ['Warengruppe', 'Wochenende']

#Schauen welchen Typ die gewählten Features haben, deren Einträge checken und in kategorische umwandeln
print(dataf[categorical_features].dtypes)
print("Unique Values:\n",dataf[categorical_features].apply(lambda x: x.unique()))
for col in categorical_features:
    dataf[col] = dataf[col].astype('category')

# Die gewählten Features hier in dummies codieren
features = pd.get_dummies(dataf[categorical_features], drop_first=True, dtype=int)

# hier könnte man jetzt nicht kategroische numerische Features mit anfügen, davon haben wir glaube ich keine
#features['Spaltenname'] = dataf['Spaltenname']

# Hier zieht man jetzt die dependend Varibale aus dem ursprünglichen Datensatz raus (Bei uns der Umsatz) und fügt sie mit den Features zusammen
prepared_data = pd.concat([dataf[['Umsatz']], features], axis=1)

# missing values werden entfernt
prepared_data = prepared_data.dropna()

# Display the shape of the prepared data set
print(prepared_data.shape)
# Display the first few rows of the prepared data set
prepared_data.head()

Datum                   datetime64[ns]
KielerWoche                    float64
Warengruppe                   category
Umsatz                         float64
Bewoelkung                     float64
Temperatur                     float64
Windgeschwindigkeit            float64
Wettercode                     float64
Beschreibung                    object
FerienSH                       float64
Feiertag                       float64
Uhrzeit                         object
Heim_Auswärts                   object
Umschlag                       float64
Weihnachtsmarkt                float64
Regen                            int64
Wochentag_MDMDFSS                int64
Wochenende                    category
Jahreszeit_FSHW                  int64
Temperatur_Kategorie            object
dtype: object
Warengruppe      Datum         1.0         2.0         3.0         4.0  \
0           2013-07-01  148.828353  535.856285  201.198426   65.890169   
1           2013-07-02  159.793757  546.780787  265.261254

/tmp/ipykernel_52958/2345656826.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = dataf.pivot_table(index='Datum', columns='Warengruppe', values='Umsatz').reset_index()


,Umsatz,Warengruppe_2.0,Warengruppe_3.0,Warengruppe_4.0,Warengruppe_5.0,Warengruppe_6.0,Wochenende_1
547,148.828353,0,0,0,0,0,0
548,535.856285,1,0,0,0,0,0
549,201.198426,0,1,0,0,0,0
550,65.890169,0,0,1,0,0,0
551,317.475875,0,0,0,1,0,0


### Den vorbereiteten Datensatz in Trainings und Validierungsdaten teilen